In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [15]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap = market_cap.loc[df_48ind.index]


df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)


# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)


standardized_market_caps = standardized_market_caps.loc[standardized_mom.index]
standardized_BtoM = standardized_BtoM.loc[standardized_mom.index]
standardized_mom = standardized_mom.loc[standardized_mom.index]
market_weights = market_weights.loc[standardized_mom.index]
df_48ind = df_48ind.loc[standardized_mom.index]

df4 = df_48ind.shift(-1).dropna()

standardized_market_caps = standardized_market_caps.loc[df4.index]
standardized_BtoM = standardized_BtoM.loc[df4.index]
standardized_mom = standardized_mom.loc[df4.index]
market_weights = market_weights.loc[df4.index]
df_48ind = df_48ind.loc[df4.index]

def Expanding_window_optimization(MC, BM, MOM, r, gamma=5):
    
    def u(x, gamma=5):
        return (1+x) ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma=5):
        wi_t = market_weights
        return -(1/T) * np.sum(u(np.sum((wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)) * (r),axis=1), gamma))

    optimal_betas = {}  # Define optimal_betas here to store the optimal coefficients for each period
    
    start_year = df_48ind.index.min() # Date de début fixée à la première date dans les données de rendement
    
    # Date de fin initiale fixée à décembre 1973
    initial_end_year = pd.to_datetime('1973-12-01')
    # Date de fin courante, commence par initial_end_year et sera ajustée dans la boucle
    current_end_year = initial_end_year

    while current_end_year <= df_48ind.index.max(): # Boucle jusqu'à la dernière date dans les données de rendement 
        
        # Extraire les composantes nécessaires pour l'optimisation des coefficients pour la période sélectionnée dans la window
        MC = standardized_market_caps.loc[start_year:current_end_year]
        BM = standardized_BtoM.loc[start_year:current_end_year]
        MOM = standardized_mom.loc[start_year:current_end_year]
        r = df_48ind.loc[start_year:current_end_year]
            
        N = df_48ind.shape[1]
        T = df_48ind.shape[0]

        # Initial guess for theta
        initial_beta = np.array([0.3, 0.3, 0.3])

        # Run the optimization for the current period
        result = minimize(function_to_maximize, initial_beta, args=( MC, BM, MOM, r, T, N, gamma), method='SLSQP')

        # Store the optimal theta for the current period
        optimal_betas[current_end_year] = result.x if result.success else None

        # Élargir la fenêtre pour la prochaine itération de window en ajoutant 12 mois à la dernière date courante de fin 
        current_end_year += pd.DateOffset(months=12)

    return optimal_betas


In [16]:
optimal_betas = Expanding_window_optimization(standardized_market_caps, standardized_BtoM, standardized_mom, df_48ind)

In [21]:
optimal_betas = pd.DataFrame(optimal_betas).T

optimal_betas

,0,1,2
1973-12-01,-1.440307,0.342557,-0.705870
1974-12-01,-1.436313,0.352228,-0.694717
1975-12-01,-1.521486,0.461259,-0.456966
1976-12-01,-1.573961,0.509060,-0.338681
1977-12-01,-25734.946429,33293.935763,-28786.242992
1978-12-01,-25735.000046,33294.063087,-28786.338545
1979-12-01,-25735.000066,33294.063124,-28786.338547
1980-12-01,-25735.000230,33294.063494,-28786.338522
1981-12-01,-25734.999637,33294.062406,-28786.338372
1982-12-01,-25734.999690,33294.062354,-28786.338344
